In [1]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine, MetaData, text, update
from smartsheet_dataframe import get_as_df
import sqlite3
from sqlalchemy.dialects.sqlite import insert

In [2]:
def insert_on_conflict_nothing(table, conn, keys, data_iter):
    # "a" is the primary key in the "conflict_table"
    data = [dict(zip(keys, row)) for row in data_iter]
    stmt = insert(table.table).values(data).on_conflict_do_nothing(index_elements=["uniqueid"])
    result = conn.execute(stmt)
    return result.rowcount

In [3]:
client_pre_imp = pd.read_excel("./data/bjt_trial_tables.xlsx", sheet_name='client_pre_imp')
ga_pre_imp = pd.read_excel("./data/bjt_trial_tables.xlsx", sheet_name='ga_pre_imp')
pre_imp_plot = pd.read_excel("./data/bjt_trial_tables.xlsx", sheet_name='pre_imp_plot')
expected_harvest = pd.read_excel("./data/bjt_trial_tables.xlsx", sheet_name='expected_harvest')
ticket_id = pd.read_excel("./data/bjt_trial_tables.xlsx", sheet_name='ticket_id')

In [4]:
# Column fixing bits
# Fix data typing issues
ga_pre_imp['guidance_expiration'] = ga_pre_imp['guidance_expiration'].dt.date
ga_pre_imp['guidance_presented'] = ga_pre_imp['guidance_presented'].dt.date

# Fix naming issues
client_pre_imp.rename(columns={"cmp": "comp"},inplace=True)

In [5]:
# Define your connection to the sqlite db - if it doesn't exist it gets created
proj_code = "P:\\Temp\\harvest_db_trial\\harvest_db_trial"
conn = sqlite3.connect(f'{proj_code}.db')

In [ ]:
# Try out adding in any new data from client_preimp
# find existing keys
client_pre_imp_existing_ids = pd.read_sql_query(f'SELECT uniqueid FROM client_preimp', conn)
# existing keys in both data sets
new_rows = client_pre_imp[~client_pre_imp['uniqueid'].isin(client_pre_imp_existing_ids['uniqueid'])]
# add in new data to the database else do nothing
if not new_rows.empty:
    new_rows.to_sql('client_preimp', conn, if_exists='append', index=False)
del new_rows

In [ ]:
# Try adding ga_pre_imp data to the db
# get uniqueids already in db table
ga_pre_imp_existing_ids = pd.read_sql_query(f'SELECT uniqueid FROM ga_preimp', conn)
# get the rows that are in the excel file but not in the db table
new_rows = ga_pre_imp[~ga_pre_imp['uniqueid'].isin(ga_pre_imp_existing_ids['uniqueid'])]
# add in the new data
if not new_rows.empty:
    new_rows.to_sql('ga_preimp', conn, if_exists='append', index=False)
del new_rows

### Strategize

Ok, I think there are a number of ways that we can streamline this process.

1. map out each table's unique id and set up a function that does the above addition of "new_rows"
2. figure out how to prevent duplication of data in ticketid table and plotid table

In [6]:
# 0.5
ref_dict = {"client_preimp": [client_pre_imp, 'uniqueid'],
            "ga_preimp": [ga_pre_imp, "uniqueid"],
            "expected_harvest": [expected_harvest, "ticketid"],
            "preimp_plot": [pre_imp_plot, ['uniqueid', 'plotid']],
            "ticketid": [ticket_id, ['uniqueid', 'ticketid']]}

In [11]:
# 1. 
def add_new_rows(df, table, conn, id):
    if len(id) == 2:
        try:
            existing_ids = pd.read_sql_query(f'SELECT {id[0]}, {id[1]} from {table}', conn)
            merged = df.merge(existing_ids, how='left', indicator=True)
            new_rows = merged[merged['_merge'] == 'left_only'].drop(columns=['_merge'])
        except:
            print(f"Something went wrong with adding new rows to {table}")
    else:
        try:    
            existing_ids = pd.read_sql_query(f'SELECT {id} from {table}', conn)
            new_rows = df[~df[f'{id}'].isin(existing_ids[f'{id}'])]
        except:
            print(f"Something went wrong with adding new rows to {table}")    
    
    
    if not new_rows.empty:
        new_rows.to_sql(table, conn, if_exists='append', index=False)
    print(table, len(new_rows))
    del new_rows
    

In [13]:
i = 0
for key in ref_dict.keys():
    print(i)
    add_new_rows(ref_dict[key][0], key, conn, ref_dict[key][1])
    i+=1

0
client_preimp 0
1
ga_preimp 0
2
expected_harvest 0
3
preimp_plot 0
4
ticketid 0


In [ ]:
conn.close()

In [14]:
bugger = {"numbers": [1, 2, 3, 4, 5, 6],
          "letters": ["a", "b", "c", "d", "e", 'f']
}
bugger1 = {"numbers": [1, 2, 7, 8, 9, 10],
          "letters": ["a", "b", "g", "h", "i", 'j']
}

dfa =pd.DataFrame(bugger)
dfb = pd.DataFrame(bugger1)

In [15]:
len(dfa)

6

In [21]:
ranger = [i for i in range(len(dfa))]


5